# Keypoint Alignment Morphing

In [1]:
import os

os.chdir("../..")

In [2]:
!pwd

/Users/wiktorlazarski/Desktop/my_project/face-morphing


In [3]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (12, 10)

In [4]:
F_SAMPLE_ANN = os.path.join("data", "morph_samples", "1.txt")
F_SAMPLE_IMG = os.path.join("data", "morph_samples", "1.jpg")

S_SAMPLE_ANN = os.path.join("data", "morph_samples", "2.txt")
S_SAMPLE_IMG = os.path.join("data", "morph_samples", "2.jpg")

In [5]:
def load_sample(ann_file, img_file):
    keypoints = []

    with open(ann_file, "r") as ann_file:
        for i, line in enumerate(ann_file.readlines()):
            if i == 0:
                continue
        
            coords = [float(val) for val in line.split(",")]
            keypoints.append(tuple(coords))
    
    img = cv2.imread(img_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    return img, keypoints

In [6]:
f_img, f_kps = load_sample(F_SAMPLE_ANN, F_SAMPLE_IMG)
s_img, s_kps = load_sample(S_SAMPLE_ANN, S_SAMPLE_IMG)

In [10]:
f_h, f_w = f_img.shape[:2]
s_resized = cv2.resize(s_img, (f_w, f_h))
s_h, s_w = s_img.shape[:2]
sx, sy = f_w / s_w, f_h / s_h
new_s_kps = []

for kp in s_kps:
    x, y = kp
    new_x = x * sx
    new_y = y * sy
    new_s_kps.append((round(new_x, 2), round(new_y, 2)))

In [17]:
out = cv2.findHomography(np.array(f_kps), np.array(new_s_kps))

In [19]:
h_mat, status = out

In [20]:
h_mat

array([[ 6.61138531e-01, -4.29275601e-01,  6.66195085e+02],
       [ 1.10545590e-01,  3.58997806e-01,  1.11902948e+02],
       [ 8.34051551e-06, -2.12059337e-04,  1.00000000e+00]])

In [24]:
?cv2.warpPerspective

Docstring:
warpPerspective(src, M, dsize[, dst[, flags[, borderMode[, borderValue]]]]) -> dst
.   @brief Applies a perspective transformation to an image.
.   
.   The function warpPerspective transforms the source image using the specified matrix:
.   
.   \f[\texttt{dst} (x,y) =  \texttt{src} \left ( \frac{M_{11} x + M_{12} y + M_{13}}{M_{31} x + M_{32} y + M_{33}} ,
.        \frac{M_{21} x + M_{22} y + M_{23}}{M_{31} x + M_{32} y + M_{33}} \right )\f]
.   
.   when the flag #WARP_INVERSE_MAP is set. Otherwise, the transformation is first inverted with invert
.   and then put in the formula above instead of M. The function cannot operate in-place.
.   
.   @param src input image.
.   @param dst output image that has the size dsize and the same type as src .
.   @param M \f$3\times 3\f$ transformation matrix.
.   @param dsize size of the output image.
.   @param flags combination of interpolation methods (#INTER_LINEAR or #INTER_NEAREST) and the
.   optional flag #WARP_INVERSE_MAP, th